In [5]:
import os, cv2, random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm    #Helps in vis
from random import shuffle 

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
import gc

Using TensorFlow backend.


In [30]:
from keras.applications.nasnet import NASNetLarge, preprocess_input

In [25]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle

In [4]:
import tensorflow as tf

In [49]:
TRAIN_FOLDER = '../../Human_Protein_Atlas/input/train/'
TEST_FOLDER =  '../../Human_Protein_Atlas/input/test/'
IMG_SIZE = 331

In [7]:
train_image_list = os.listdir(TRAIN_FOLDER)
test_image_list = os.listdir(TEST_FOLDER)

In [8]:
len(test_image_list)

46808

In [9]:
len(train_image_list)

124288

In [10]:
sample_submission = pd.read_csv('../../Human_Protein_Atlas/input/sample_submission.csv')

In [11]:
sample_submission.shape

(11702, 2)

In [14]:
data = pd.read_csv('../../Human_Protein_Atlas/input/train.csv')

In [15]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [16]:
train_image_list[0]

'3b7af758-bbb8-11e8-b2ba-ac1f6b6435d0_blue.png'

In [51]:
image_channels = ['_red.png', '_yellow.png', '_green.png', '_blue.png']

def load_image(path, dropped_channel):
    image_red_ch = Image.open(path+'_red.png')
    image_yellow_ch = Image.open(path+'_yellow.png')
    image_green_ch = Image.open(path+'_green.png')
    image_blue_ch = Image.open(path+'_blue.png')
    image = np.stack((
    np.array(image_red_ch), 
    np.array(image_green_ch), 
    np.array(image_blue_ch)), -1)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    return image

In [65]:
image_channels = ['_red.png', '_yellow.png', '_green.png', '_blue.png']


In [72]:
list(set(image_channels) - set(['_red.png']))

['_yellow.png', '_green.png', '_blue.png']

In [73]:

def load_image(path, dropped_channel):
    channels_to_use = list(set(image_channels) - set([dropped_channel]))
    image_one_ch = Image.open(path+channels_to_use[0])
    image_two_ch = Image.open(path+channels_to_use[1])
    image_three_ch = Image.open(path+channels_to_use[2])
    image = np.stack((
    np.array(image_one_ch), 
    np.array(image_two_ch), 
    np.array(image_three_ch)), -1)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    return image

In [74]:
image_path = TRAIN_FOLDER + data.Id.values[0]

In [53]:
image_path

'../../Human_Protein_Atlas/input/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0'

In [75]:
image_0 = load_image(image_path, '_yellow.png')

In [76]:
image_0.shape

(331, 331, 3)

In [56]:
img = np.expand_dims(image_0, axis=0)
img = preprocess_input(img)

In [57]:
img.shape

(1, 331, 331, 3)

In [36]:
img = preprocess_input(img)

In [59]:
img.min()

-1.0

In [47]:
base_model = NASNetLarge(weights='imagenet',
                   input_shape=((331,331, 3)))

359751680/359748576 [==============================] - 37s 0us/step


In [50]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('global_average_pooling2d_1').output)



In [43]:
model2 = VGG19()

574717952/574710816 [==============================] - 27s 0us/step


In [60]:
img = model.predict(img).flatten()

In [62]:
img.shape

(4032,)

In [ ]:
for Id in tqdm(data.Id.values):
    for dropped_channel in image_channels:
        image_path = TRAIN_FOLDER + Id
        img = load_image(image_path, dropped_channel)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        img = model.predict(img).flatten()
        np.save('../../Human_Protein_Atlas/numpy_files/train/'+Id+'_missing'+dropped_channel, img)

 15%|█▌        | 4791/31072 [26:09<2:29:42,  2.93it/s]

In [64]:
image_path

'../../Human_Protein_Atlas/input/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0'